# ROB2004 Final Project - manipulation of objects

The goal of this project is to solve a simple manipulation task.

## Instructions
* Solve the problem below.
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a report (maximum 3 pages) in pdf format detailing the methodology you followed to solve the problem. You should add plots in the report (does not count for the page limit).
* This is an individual project (dual submission of code/report is not allowed)

We will use a model of the Frank-Emika Panda robot. This robot has 7 revolute joints and its kinematics is described in the picture below.

To run this project, you need to install Mujoco and Pinocchio. Please reach out as soon as possible for help if you do not know how to do it.

<div>
<img src="./panda.jpg" width="200"/>
</div>

# Problem

Implement controller(s) using the code skeleton below to get the robot to put the red cube inside the bowl and the blue cube on top of the tower of green cubes (without breaking the tower!).

<div>
<img src="./manipulation_scene.jpg" width="400"/>
</div>

You are free to use the methods that you want to solve the task, with the following constraints:
* You cannot use any external library apart from numpy, scipy, cvxopt or proxQP
* You need to use at least one task-space controller
* You need to use at least one joint-space controll (with an inverse geometry method)
* You need to compensate for the gravity of the robot

In your report, describe the controllers you designed and justify your choices. Analyze the behavior of the system (include plots of the end-effector trajectories, velocities, joint trajectories, etc as you see fit). Do not copy-paste code in the report.

## Scene details
* The bowl is located 25cm forward and 40cm right of the robot.
* The green tower is located 35cm forward and 50cm to the left of the robot
* The blue cube is located 45cm forward and 20cm to the left of the robot
* The red cube is located 50cm in front of the robot
* The x axis of the robot base is pointing foward and its y axis is pointing to its left


In [ ]:
# setup some librairies and display options
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt

import time

# be careful we will only print the first 4 digits and round small numbers in arrays
np.set_printoptions(suppress=True, precision=4)

# this library contains the simulation and helper functions for the robot
from MujocoSim import FR3Sim

In [ ]:
# this creates the simulation environment
my_robot = FR3Sim()

In [ ]:
## the controller is run every 1ms

## sample control loop running for 5 seconds (i.e. 5/0.001 = 5000 steps)
num_steps = 5000

for i in range(num_steps):
    # this function gets the joint positions and velocities
    q, dq = my_robot.get_state()

    # this function computes the foward kinematics (given joint positions)
    # it returns an homogeneous transform
    T = my_robot.get_pose(q)
    
    # this function computes the endeffector Jacobian (given joint positions)
    # the Jacobian maps joint velocities to endeffector twist (in endfefector frame! not worldframe)
    jacobian = my_robot.get_jacobian(q)

    ## this function get the gravity vector
    gravity = my_robot.get_gravity(q)

    # a damping controller and instruction to close and open the gripper
    u = -40*dq
    if i < num_steps/2:
        gripper_command = -10. # this will close the gripper
    else:
        gripper_command = 10. # this will open the gripper

    ## this function sends joint torques and 
    ## commands opening or closing of the gripper (positive number applies an opening force)
    ## it will also advance the simulation by 1ms
    my_robot.send_joint_torque(u, gripper_command)
    time.sleep(0.001) # we wait 0.001 seconds to simulate the 1ms control loop
